The derivation of new information based on the existing occurs in Tier 2 of Ampel. In optical astronomy, the typical base available dataset is a `LightCurve`. From this we typically wish to calculate features like rise-time, peak brightness and age. More complex but funtionally identical tasks involve ML-based classification. 

To make use of the AMPEL capabilities for large scale feature calculation, provenance and real-time reaction, such analysis units need to be expressed as python modules inheriting from the proper base class. This ensures that AMPEL knows which kind of data the method expects as well as what sort of output is expected.

Additional notes:
- T2 units are either _state bound_, in which case they will be called each time a new datapoint is obtained, or _datapoint bount_ in which case it is only called the first time a transient is detected. This notebook deals with state bound case.
- T2 units can make use also of information derived from other T2 units. An example of this would be using the redshift as obtained by a previous unit which looked through catalogs for distance information. Such units are called _linked_. This notebook demonstrates a non-linked T2.

##### 1. Obtaining a sample lightcurve
This section will obtain the `LightCurve` of a sample SN (see the "Demo - Accessing the ZTF Alert Archive" notebook).

In [ ]:
import os
from ampel_notebook_utils import api_get_lightcurve

Access to both the AMPEL-DESY data archive as well as any currently running live instances are maintained through a system of personal tokens, authorized through your github account. Tokens can be retrieved through:
https://ampel.zeuthen.desy.de/live/dashboard/tokens 

As we are here accessing the archive, the (permanent until changed) archive token is used (select this tab). As this does not change it can conveniently be stored as a local variable. 

[Accessing a live instance requires a temporary token, and is not discussed in this notebook.]

In [ ]:
archivetoken = "MYARCHIVETOKEN" 

In [ ]:
# ZTF name of transients to explore
snname = "ZTF22aaylnhq"

In [ ]:
# Obtain the lightCurve
lightCurve = api_get_lightcurve(snname, archivetoken)

In [ ]:
# The content of the transient datapoint can be directly printed...
lightCurve.photopoints[0]

In [ ]:
# ... or obtain a tuple of a select subset 
gphot = lightCurve.get_ntuples(['jd', 'magpsf'], 
                              {'attribute': 'fid', 'operator': '==', 'value': 1})
gphot[0:5]

##### 2. Developing an analysis

We will here develop a dummy lightcurve analysis looking for position drift with time. In a real case this would replaced by a science driven calculation.

In [ ]:
# The algorithm behaviour will be controlled by a set of parameters.
# These are fixed by the user in production to specficy the expected behaviour.
use_filters = [1]

In [ ]:
posdata = lightCurve.get_ntuples(['jd', 'magpsf', 'ra', 'dec', 'fid']) 

In [ ]:
posdata = [p for p in posdata if p[-1] in use_filters]
print(f'Working with {len(posdata)} datapoints')

In [ ]:
# We can make use of standard python packages, as well as non-standard when motivated
import numpy as np
posdata = np.asarray(posdata)

In [ ]:
jd = posdata[:,0]
ra_arcsec = (posdata[:,2]-posdata[:,2].mean())*60*60
dec_arcsec = (posdata[:,3]-posdata[:,3].mean())*60*60

In [ ]:
from numpy.polynomial.polynomial import Polynomial

In [ ]:
f = Polynomial.fit(jd, ra_arcsec, 1)
ra_slope = f.coef[1]
f = Polynomial.fit(jd, dec_arcsec, 1)
dec_slope = f.coef[1]
t_span = jd.max()-jd.min()

In [ ]:
print('Found slope in RA {} and in Dec {} arcsec / day for {} datapoints during {} days'.format(
                                    ra_slope, dec_slope, len(jd), t_span))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(jd, ra_arcsec,'o', label='RA')
plt.plot(jd, dec_arcsec,'o', label='Dec')
plt.xlabel('JD')
plt.ylabel('Shift w.r.t. mean position (arsec)')
_ = plt.legend()

##### 3. Develop as an Ampel unit

We will here create an Ampel class carrying out the same calculation as above.

Notes:
- We encourage type hinting.
- The output consists in a dictionary, which is then made available for the next tier (or linked T2 unit). This could involve a decision as to whether the slope is significant enough to react to.

In [ ]:
# The base lightcurve T2 unit is a realization of an AbsLightCurveT2Unit
from ampel.abstract.AbsLightCurveT2Unit import AbsLightCurveT2Unit

In [ ]:
AbsLightCurveT2Unit??

In [ ]:
# We here summarize imports done above, to mirror what would exist in a standalone file.
from typing import Optional

import os
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from ampel.abstract.AbsLightCurveT2Unit import AbsLightCurveT2Unit
from ampel.view.LightCurve import LightCurve

# Unit output verified through one of these types
from ampel.types import UBson
from ampel.struct.UnitResult import UnitResult




class T2DemoCoordEval(AbsLightCurveT2Unit):
    """
    Fit a linear evolution to the RA and Dec coordinates of a LightCurve. 
    
    Parameters:
     *use_filters* lists the filter ids, as encoded in the datapoint "fid" field
    (e.g. present in ZTF alerts).
    
    Optionally plot debug lightcurves
     *plot_lc*
     *plot_lc_dir* 
    
    """

    # These parameters can be provided by the user when specifying the channel
    # If no (default) value exists this has to be provided
    use_filters: list[int] = [1, 2, 3]
        
    plot_evo: bool = False
    plot_evo_dir: Optional[str]

    def process(self, light_curve: LightCurve) -> UBson | UnitResult:
        """        
        The process method is called for each Transient state belonging to the transient.        
        """
        
        # Extract residuals in required bands
        posdata = [p for p in lightCurve.get_ntuples(['jd', 'magpsf', 'ra', 'dec', 'fid']) 
                       if p[-1] in self.use_filters]
        posdata = np.asarray(posdata)
        jd = posdata[:,0]
        ra_arcsec = (posdata[:,2]-posdata[:,2].mean())*60*60
        dec_arcsec = (posdata[:,3]-posdata[:,3].mean())*60*60
        
        # Each AMPEL inut is provided with a logger instance which can be used
        # to store run log info for later processing.
        self.logger.info('Info level output should be used sparingly as it creates large datastreams.') 
        self.logger.debug('Debug output useful in development.')
        self.logger.debug('Data expected to be used later is most efficiently saved as unit output.')
        self.logger.debug( f'Working with {len(jd)} datapoints' )

        # Carry out fits
        f = Polynomial.fit(jd, ra_arcsec, 1)
        ra_slope = f.coef[1]
        f = Polynomial.fit(jd, dec_arcsec, 1)
        dec_slope = f.coef[1]
        t_span = jd.max()-jd.min()                
                         
        # Construct output dictionary
        t2_output = {'RA_slope': ra_slope, 'Dec_slope': dec_slope, 't_span': t_span, 'ndet': len(jd) }
        
        # Optionally make debug plot 
        if self.plot_evo and self.plot_evo_dir is not None:
            
            plt.figure()
            plt.plot(jd, ra_arcsec,'o', label='RA')
            plt.plot(jd, dec_arcsec,'o', label='Dec')
            plt.xlabel('JD')
            plt.ylabel('Shift w.r.t. mean position (arsec)')
            plt.legend()
            plt.savefig( os.path.join(self.plot_evo_dir, 'plot_T2DemoCoordEval.png') )
            plt.close()
            


        return t2_output

##### 4. Initialize and run the T2 unit

We will here reprocess the LightCurve using the unit. The logging unit is here directly shown.

In [ ]:
from ampel.log import AmpelLogger

In [ ]:
t2instance = T2DemoCoordEval(logger=AmpelLogger.get_logger())

In [ ]:
t2output = t2instance.process(lightCurve)

In [ ]:
# This is the result that would be stored in the AMPEL DB, and provided to downstream units.
t2output

Redo with different parameters:

In [ ]:
t2other = T2DemoCoordEval(use_filters=[1], plot_evo=True, plot_evo_dir='.', logger=AmpelLogger.get_logger())

In [ ]:
t2other.process(lightCurve)

In [ ]:
from IPython import display
display.Image("./plot_T2DemoCoordEval.png")

##### 5. Make a unit available for real-time, archive processing, or to other users.

These steps all involve making the unit designed above accessible to other users (whether local users or data centers). This is best done through a dedicated github repository where:
- The above python method is stored in a file `T2DemoCoordEval.py` placed in an ampel/contrib/[repositoryID]/t2 directory. 
- The unit name (`T2DemoCoordEval`) added to the conf/[repositoryName]/unit.yml file. 

Other users can then clone and install the repository, after which the unit will be accessible and can be requsted in the channel jobfile.

Example setup can be seen in this repository (Ampel-HU-astro). 